In [1]:
import os
os.makedirs('output', exist_ok=True)

import sys
sys.path.append('../')
from utils import load_walksheds, overlay_wks

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt
import fiona
import os

In [2]:
wksp5, wksp75 = load_walksheds()
all_boundaries= gpd.read_file(r"../../Data/MD-DC-VA Boundaries.zip")

In [3]:
Table_gdb_MD = fiona.listlayers(r"../../Data/GDB states/tlgdb_2021_a_24_md.gdb")

In [4]:
Table_gdb_MD

['Incorporated_Place',
 'County',
 'Census_Designated_Place',
 'Block_Group',
 'County_Subdivision',
 'Census_Tract',
 'Block20']

In [5]:
## block group geodataframe MD, DC, VA

gdb_MD_BG = gpd.read_file(r"../../Data/GDB states/tlgdb_2021_a_24_md.gdb", driver='FileGDB', layer='Block_Group')

gdb_DC_BG = gpd.read_file (r"../../Data/GDB states/tlgdb_2021_a_11_dc.gdb", driver='FileGDB', layer='Block_Group')

gdb_VA_BG = gpd.read_file (r"../../Data/GDB states/tlgdb_2021_a_51_va.gdb", driver='FileGDB', layer='Block_Group')

In [6]:
## read enrollment layer of ACS ##
ACS_enrollment_MD = gpd.read_file(r"../../Data/ACS_2020_5YR_BG_24_MARYLAND.gdb", driver='FileGDB', layer='Enrollment')
ACS_enrollment_DC = gpd.read_file(r"../../Data/ACS_2020_5YR_BG_11_DISTRICT_OF_COLUMBIA.gdb", driver='FileGDB', layer='Enrollment')
ACS_enrollment_VA = gpd.read_file(r"../../Data/ACS_2020_5YR_BG_51_VIRGINIA.gdb",driver='FileGDB', layer='Enrollment')

In [7]:
ACS_enrollment_MD.head()

,OBJECTID__,GEOID,No_Vehicle,High_School,Graduate,geometry
0,97767.0,15000US240010001001,0.0,8.0,20.0,None
1,97768.0,15000US240010001002,33.0,346.0,40.0,None
2,97769.0,15000US240010001003,30.0,137.0,0.0,None
3,97770.0,15000US240010002001,0.0,107.0,36.0,None
4,97771.0,15000US240010002002,0.0,103.0,32.0,None


In [8]:
ACS_enrollment_MD['GEOID'] = ACS_enrollment_MD['GEOID'].str.replace('15000US', '')
ACS_enrollment_DC['GEOID'] = ACS_enrollment_DC['GEOID'].str.replace('15000US', '')
ACS_enrollment_VA['GEOID'] = ACS_enrollment_VA['GEOID'].str.replace('15000US', '')

In [9]:
## join ACS to BG of three MD, DC, VA
ACS_Enrollment_MD = pd.merge(ACS_enrollment_MD, gdb_MD_BG, on='GEOID', how='outer')

ACS_Enrollment_DC = pd.merge(ACS_enrollment_DC, gdb_DC_BG, on='GEOID', how='outer')

ACS_Enrollment_VA = pd.merge(ACS_enrollment_VA, gdb_VA_BG, on='GEOID',how='outer')

In [10]:
ACS_Enrollment_VA.rename(columns = {'geometry_y':'geometry'}, inplace = True)
ACS_Enrollment_DC.rename(columns = {'geometry_y':'geometry'}, inplace = True)
ACS_Enrollment_MD.rename(columns = {'geometry_y':'geometry'}, inplace = True)

ACS_Enrollment_VA = ACS_Enrollment_VA.drop(columns=['geometry_x'])
ACS_Enrollment_DC = ACS_Enrollment_DC.drop(columns=['geometry_x'])
ACS_Enrollment_MD = ACS_Enrollment_MD.drop(columns=['geometry_x'])

In [11]:
ACS_Enrollment_MD.head(2)

,OBJECTID__,GEOID,No_Vehicle,High_School,Graduate,NAMELSAD,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,97767.0,240010001001,0.0,8.0,20.0,Block Group 1,114609102.0,2207428.0,+39.6704707,-78.3878895,"MULTIPOLYGON (((-78.49625 39.60436, -78.49612 ..."
1,97768.0,240010001002,33.0,346.0,40.0,Block Group 2,215159992.0,508136.0,+39.6796402,-78.5461225,"MULTIPOLYGON (((-78.66648 39.64913, -78.66643 ..."


In [12]:
ACS_Enrollment1 = pd.merge(ACS_Enrollment_MD, ACS_Enrollment_DC, how='outer')

In [13]:
ACS_enrollment = pd.merge(ACS_Enrollment1, ACS_Enrollment_VA, how='outer')

In [15]:
ACS_enrollment = ACS_enrollment.to_crs('EPSG:4326')

In [16]:
from geopandas.tools import overlay

intp5, intp75 = overlay_wks(ACS_enrollment)

In [17]:
#sum_enrollment_stations= intersect_enrollment_stations.groupby(["STATIONNM"])['Aggregate household enrollment'].sum()
sum_HighSchool_stations_p5 = intp5.groupby(["Name_1"])["High_School"].median().reset_index()
sum_HighSchool_stations_p75 = intp75.groupby(["Name_1"])["High_School"].median().reset_index()

sum_Graduate_stations_p5 = intp5.groupby(["Name_1"])["Graduate"].median().reset_index()
sum_Graduate_stations_p75 = intp75.groupby(["Name_1"])["Graduate"].median().reset_index()

In [20]:
sum_HighSchool_stations_p75.head()

,Name_1,High_School
0,ADDISON ROAD-SEAT PLEASANT,177.0
1,ANACOSTIA,NaN
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,NaN
3,ARLINGTON CEMETERY,NaN
4,Ashburn,NaN


In [19]:
sum_Graduate_stations_p5.head()

,Name_1,Graduate
0,ADDISON ROAD-SEAT PLEASANT,32.0
1,ANACOSTIA,NaN
2,ARCHIVES-NAVY MEMORIAL-PENN QUARTER,NaN
3,ARLINGTON CEMETERY,NaN
4,Ashburn,NaN


In [22]:
sum_HighSchool_stations_p5.rename(columns={'High_School':'High_School_p5'}).to_csv(r"output/High_school_student_population_Enrollment_p5.csv", index=False)
sum_HighSchool_stations_p75.rename(columns={'High_School':'High_School_p75'}).to_csv(r"output/High_school_student_population_Enrollment_p75.csv", index=False)

sum_Graduate_stations_p5.rename(columns={'Graduate':'Graduate_p5'}).to_csv(r"output/grad_student_population_Enrollment_p5.csv", index=False)
sum_Graduate_stations_p75.rename(columns={'Graduate':'Graduate_p75'}).to_csv(r"output/grad_student_population_Enrollment_p75.csv", index=False)